In [23]:
# Importing modules
import glob
import os
from pathlib import Path
import pandas as pd

In [103]:
lista = ["5787544", "5787545", "5787546", "5787547", "5787548", "5787549", "5787550", "1234567", "5990000", "5990001", "5990002"]

In [85]:
def FileFinder(YKR_IDs: list, input_folder_name: str, to_file=False):
    input_folder = Path(input_folder_name).absolute()
    counter = 1
    text_files = []
    for root, dirs, files in os.walk(input_folder):
        for name in files:
            if "checkpoint" in name:
                    continue
            split = name.split(" ")
            num = split[1]
            ID = num[:-4]
            if ID in YKR_IDs: 
                print(f"Processing file {name}. Progress: {counter}/{len(YKR_IDs)}")
                counter += 1
              
              
              
              

In [86]:
FileFinder(lista, "input_data")

Processing file travel_times_to_ 5787544.txt. Progress: 1/11
Processing file travel_times_to_ 5787545.txt. Progress: 2/11
Processing file travel_times_to_ 5787546.txt. Progress: 3/11
Processing file travel_times_to_ 5787547.txt. Progress: 4/11
Processing file travel_times_to_ 5787548.txt. Progress: 5/11
Processing file travel_times_to_ 5787549.txt. Progress: 6/11
Processing file travel_times_to_ 5787550.txt. Progress: 7/11
Processing file travel_times_to_ 5990000.txt. Progress: 8/11
Processing file travel_times_to_ 5990001.txt. Progress: 9/11
Processing file travel_times_to_ 5990002.txt. Progress: 10/11


In [117]:
def FileFinder2(YKR_IDs: list, input_folder_name: str, to_file=False):
    input_folder = Path(input_folder_name).absolute()
    counter = 1
    text_files = []
    for root, dirs, files in os.walk(input_folder):
        for name in files:
            if "checkpoint" in name:
                continue
            text_files.append(name)
    
    for YKR_ID in YKR_IDs:
        id_check = f"travel_times_to_ {YKR_ID}.txt"
        if id_check not in text_files:
            print(f"YKR_ID {YKR_ID} does not exist in the input folder!")
        else:
            print(f"Processing file travel_times_to_ {YKR_ID}.txt. Progress: {counter}/{len(text_files)}")
            counter += 1
    
    if to_file == True:
        with open("YKR_ID_fps", "w") as output:
            output.write(str(text_files))
        print("Wrote the filepaths to YKR_ID_fps text file")
    
    return text_files
    
        
              

In [118]:
FileFinder2(lista, "input_data", to_file=True)


Processing file travel_times_to_ 5787544.txt. Progress: 1/10
Processing file travel_times_to_ 5787545.txt. Progress: 2/10
Processing file travel_times_to_ 5787546.txt. Progress: 3/10
Processing file travel_times_to_ 5787547.txt. Progress: 4/10
Processing file travel_times_to_ 5787548.txt. Progress: 5/10
Processing file travel_times_to_ 5787549.txt. Progress: 6/10
Processing file travel_times_to_ 5787550.txt. Progress: 7/10
YKR_ID 1234567 does not exist in the input folder!
Processing file travel_times_to_ 5990000.txt. Progress: 8/10
Processing file travel_times_to_ 5990001.txt. Progress: 9/10
Processing file travel_times_to_ 5990002.txt. Progress: 10/10
Wrote the filepaths to YKR_ID_fps text file


['travel_times_to_ 5787544.txt',
 'travel_times_to_ 5787545.txt',
 'travel_times_to_ 5787546.txt',
 'travel_times_to_ 5787547.txt',
 'travel_times_to_ 5787548.txt',
 'travel_times_to_ 5787549.txt',
 'travel_times_to_ 5787550.txt',
 'travel_times_to_ 5990000.txt',
 'travel_times_to_ 5990001.txt',
 'travel_times_to_ 5990002.txt']

In [20]:
import glob
for filename in glob.glob('*.txt'):
    with open(os.path.join(os.cwd(), filename), 'r') as f:
        print(f)


In [19]:
path = r'input_data'
for filename in glob.glob(os.path.join(path, '*.txt')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        print(f)

<_io.TextIOWrapper name='C:\\Users\\zuzzuz\\Desktop\\PYQGIS_101\\AUTOGIS_PACKAGE\\final-assignment-pjustus\\input_data\\travel_times_to_ 5787544.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='C:\\Users\\zuzzuz\\Desktop\\PYQGIS_101\\AUTOGIS_PACKAGE\\final-assignment-pjustus\\input_data\\travel_times_to_ 5787545.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='C:\\Users\\zuzzuz\\Desktop\\PYQGIS_101\\AUTOGIS_PACKAGE\\final-assignment-pjustus\\input_data\\travel_times_to_ 5787546.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='C:\\Users\\zuzzuz\\Desktop\\PYQGIS_101\\AUTOGIS_PACKAGE\\final-assignment-pjustus\\input_data\\travel_times_to_ 5787547.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='C:\\Users\\zuzzuz\\Desktop\\PYQGIS_101\\AUTOGIS_PACKAGE\\final-assignment-pjustus\\input_data\\travel_times_to_ 5787548.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='C:\\Users\\zuzzuz\\Desktop\\PYQGIS_101\\AUTOGIS_PACKAGE\\final-assignment-pjustus\\in

In [27]:
list(p.glob('**/*.txt'))

[WindowsPath('input_data/travel_times_to_ 5787544.txt'),
 WindowsPath('input_data/travel_times_to_ 5787545.txt'),
 WindowsPath('input_data/travel_times_to_ 5787546.txt'),
 WindowsPath('input_data/travel_times_to_ 5787547.txt'),
 WindowsPath('input_data/travel_times_to_ 5787548.txt'),
 WindowsPath('input_data/travel_times_to_ 5787549.txt'),
 WindowsPath('input_data/travel_times_to_ 5787550.txt'),
 WindowsPath('input_data/.ipynb_checkpoints/travel_times_to_ 5787550-checkpoint.txt')]

In [40]:
input_folder = Path(input_folder_name).absolute()


C:\Users\zuzzuz\Desktop\PYQGIS_101\AUTOGIS_PACKAGE\final-assignment-pjustus\input_data


In [42]:
location = os.getcwd()
print(location)

C:\Users\zuzzuz\Desktop\PYQGIS_101\AUTOGIS_PACKAGE\final-assignment-pjustus


In [2]:
def TableJoiner():

SyntaxError: unexpected EOF while parsing (<ipython-input-2-6b3f7d4f316b>, line 1)

In [3]:
def Visualizer():

SyntaxError: unexpected EOF while parsing (<ipython-input-3-935f67ef1ef3>, line 1)

In [4]:
def ComparisonTool():

SyntaxError: unexpected EOF while parsing (<ipython-input-4-c7fae3c3a60c>, line 1)